In [ ]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from bootstrapping.bootstrapping import BootStrapping
from make_representations.representation_maker import Metagenomic16SRepresentation
from classifier.classical_classifiers import RFClassifier,SVM
from classifier.DNN import DNNMutliclass16S
import pandas as pd

# Crohn's Disease

### Ler fastaq

In [2]:
# LER OS FASTAQ

# fasta_files: arquivos fastaq
# mapping: orde na que foron lidos/iterados
#fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq')
fasta_files, mapping = FileUtility.read_fasta_directory('../../crohns_disease/samples_fastq/', 'fastq', 
                only_files=["ERR1368879.fastq", "ERR1368880.fastq", "ERR1368881.fastq", 
                            "ERR1368882.fastq", "ERR1368883.fastq", "ERR1368884.fastq",
                            "ERR1368885.fastq", "ERR1368886.fastq", "ERR1368887.fastq",
                            "ERR1368888.fastq", "ERR1368889.fastq", "ERR1368890.fastq"])

### 1. Bootstrapping

In [9]:
# CONSTRUIR OBXECTO PARA O BOOTSTRAPPING
'''
input_dir: input directory or a list of files
output_dir: a directory to generate the output files
sampling_sizes: a list of sampling sizes (N's)
n_resamples: number of resamples from each file (N_R = 10)
seqtype: file suffixes fastq or fasta etc.
M: number of files from the directory to make samplings from (M)
'''

BS=BootStrapping(fasta_files,'../../crohns_disease/results/bootstrapping/', seqtype='fastq',
                 sampling_sizes=[10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000], n_resamples=10, M=10)

In [ ]:
# add k-mer samplings
for k in [3,4,5,6,7,8]:
    print(k,'-mer bootstrapping..')
    BS.add_kmer_sampling(k)

In [19]:
# GARDAR RESULTADOS EN ARQUIVO 'bootstrapping_crohns_disease_prediction' NA RUTA QUE SE LLE INDICOU AO DEFINIR A INSTANCIA BS
BS.save_me('bootstrapping_crohns_disease_prediction')

### 2. k-mer representations

In [ ]:
# ENTRENAMENTO COS MELLORES TAMAÑOS DE K-MER (K) E OS SEUS CORRESPONDENTES TAMAÑOS DE MOSTRA (N) 
# (SEGÚN OS RESULTADOS DE BOOTSTRAPPING)

# dictionary of k-mers and their sampling values
# 6:[100,1000,2000,5000,10000,-1]  :  k-mer:[sampling values]      (-1 means using all sequences)
#sampling_dict={3:[20],4:[100],5:[500],6:[100,1000,2000,5000,10000,-1],7:[5000],8:[8000]}
sampling_dict={6:[5000]}

# number of cores
nump_p=20

# Para cada arquivo fastaq, creamos unha representación por cada posible combinación de tamaño de k-mer e tamaño de mostra, e gardamos os resultados
for k in sampling_dict.keys():  # for each k-mer
    for N in sampling_dict[k]:  # select a sampling size N
        print(k,'-mers with sampling size ',N)
        
        RS=Metagenomic16SRepresentation(fasta_files, mapping, N, nump_p)
        RS.generate_kmers_all(k, save='../../crohns_disease/datasets/PRUEBA-dataset_'+str(k)+'-mers_rate_complete1359_seq_'+str(N))

### 3. Classifers

#### Classical Classifiers

In [2]:
# CARGAR O ARQUIVO DE REPRESENTACIÓN K-MERS QUE MELLORES RESULTADOS DESE E OS RESULTADOS DE CLASIFICACIÓN REAIS

# X = matriz con datos:  (nº mostra, nº k-mers da secuencia) Abundancia relativa do k-mer na secuencia
#     en total temos 1359 mostras e 4096 6-mers
# Y = lista de resultados reais:  (CD , not CD)
#X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/dataset_6-mers_rate_complete1359_seq_5000.npz')
X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.npz')
#Y=FileUtility.load_list('../../crohns_disease/datasets/labels_disease_complete1359.txt')
Y=FileUtility.load_list('../../crohns_disease/datasets/PRUEBA-labels_disease_complete1359.txt')


# vemos os datos de X
#dataset=pd.read_csv("../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.csv")
#dataset

In [ ]:
# CREAR E ENTRENAR OS CLASIFICADORES E GARDAR OS RESULTADOS 
##### (results containing the best parameter, confusion matrix, best estimator)

################ Random Forest ################
MRF = RFClassifier(X, Y)
MRF.tune_and_eval('../../crohns_disease/results-PRUEBA/RFclassifier', n_fold=2)

In [ ]:
################ Support Vector Machine ################
MSVM = SVM(X, Y)
MSVM.tune_and_eval('../../crohns_disease/results-PRUEBA/SVMclassifier', n_fold=2)

#### DNN Classifier

In [5]:
# CARGAR O ARQUIVO DE REPRESENTACIÓN K-MERS QUE MELLORES RESULTADOS DESE E OS RESULTADOS DE CLASIFICACIÓN REAIS

#X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/dataset_6-mers_rate_complete1359_seq_5000.npz').toarray()
X=FileUtility.load_sparse_csr('../../crohns_disease/datasets/PRUEBA-dataset_6-mers_rate_complete1359_seq_5000.npz').toarray()
#Y=FileUtility.load_list('../../crohns_disease/datasets/labels_disease_complete1359.txt')
Y=FileUtility.load_list('../../crohns_disease/datasets/PRUEBA-labels_disease_complete1359.txt')

In [ ]:
# CREAR E ENTRENAR OS CLASIFICADORES E GARDAR OS RESULTADOS 
##### (results containing the best parameter, confusion matrix, best estimator)

################ Multilayer Perceptron  ################3

# DEFINIR A DNN (TIPO/ESTRUCTURA MLP)
DNN=DNNMutliclass16S(X,Y,model_arch=[1024,0.2,512,0.2,256,0.1,128,8])
#DNN=DNNMutliclass16S(X,Y,model_arch=[10,0.2,10,4])

# EXECUTAR O CROSS VALIDATION
#DNN.cross_validation('../../crohns_disease/results-PRUEBA/DNNclassifier/', gpu_dev='1', n_fold=10, epochs=100, batch_size=1000, model_strct='mlp')
DNN.cross_validation('../../crohns_disease/results-PRUEBA/DNNclassifier/', gpu_dev='1', n_fold=2, epochs=1, batch_size=1, model_strct='mlp')

# CREAR E ENTRENAR O MODELO COS PESOS ............... (ANTES: COS PESOS DO ÚLTIMO FOLD DO CV)
X_new=DNNMutliclass16S.make_activation_function('../../crohns_disease/results-PRUEBA/DNNclassifier/weights_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.37.pickle',X)

### Mostrar as métricas obtidas

In [7]:
### RF
metrics=pd.read_csv("../../crohns_disease/results-PRUEBA/RFclassifier/filtered_metrics.csv")
metrics

,Unnamed: 0,mean_test_f1_macro,mean_test_f1_micro,mean_test_precision_macro,mean_test_precision_micro,mean_test_recall_macro,mean_test_recall_micro,mean_test_roc_auc,std_test_accuracy,std_test_f1_macro,std_test_f1_micro,std_test_precision_macro,std_test_precision_micro,std_test_recall_macro,std_test_recall_micro,std_test_roc_auc
0,0,0.409524,0.500000,0.4750,0.500000,0.5625,0.500000,0.763889,0.000000,0.076190,0.000000,0.2250,0.000000,0.0625,0.000000,0.013889
1,1,0.479167,0.583333,0.4375,0.583333,0.5625,0.583333,0.645833,0.083333,0.145833,0.083333,0.1875,0.083333,0.0625,0.083333,0.020833
2,2,0.500000,0.583333,0.5000,0.583333,0.6250,0.583333,0.597222,0.083333,0.166667,0.083333,0.2500,0.083333,0.1250,0.083333,0.152778
3,3,0.409524,0.500000,0.3750,0.500000,0.5000,0.500000,0.645833,0.000000,0.076190,0.000000,0.1250,0.000000,0.0000,0.000000,0.020833


In [8]:
### SVM
metrics=pd.read_csv("../../crohns_disease/results-PRUEBA/SVMclassifier/filtered_metrics.csv")
metrics

,Unnamed: 0,mean_test_f1_macro,mean_test_f1_micro,mean_test_precision_macro,mean_test_precision_micro,mean_test_recall_macro,mean_test_recall_micro,mean_test_roc_auc,std_test_accuracy,std_test_f1_macro,std_test_f1_micro,std_test_precision_macro,std_test_precision_micro,std_test_recall_macro,std_test_recall_micro,std_test_roc_auc
0,0,0.437500,0.500000,0.483333,0.500000,0.583333,0.500000,0.770833,0.166667,0.187500,0.166667,0.316667,0.166667,0.083333,0.166667,0.104167
1,1,0.291667,0.416667,0.208333,0.416667,0.500000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.000000,0.083333,0.104167
2,2,0.291667,0.416667,0.208333,0.416667,0.500000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.000000,0.083333,0.104167
3,3,0.291667,0.416667,0.208333,0.416667,0.500000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.000000,0.083333,0.104167
4,4,0.291667,0.416667,0.208333,0.416667,0.500000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.000000,0.083333,0.104167
5,5,0.409524,0.500000,0.475000,0.500000,0.562500,0.500000,0.770833,0.000000,0.076190,0.000000,0.225000,0.000000,0.062500,0.000000,0.104167
6,6,0.409524,0.500000,0.475000,0.500000,0.562500,0.500000,0.770833,0.000000,0.076190,0.000000,0.225000,0.000000,0.062500,0.000000,0.104167
7,7,0.291667,0.416667,0.208333,0.416667,0.500000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.000000,0.083333,0.104167
8,8,0.291667,0.416667,0.208333,0.416667,0.500000,0.416667,0.770833,0.083333,0.041667,0.083333,0.041667,0.083333,0.000000,0.083333,0.104167
9,9,0.409524,0.500000,0.475000,0.500000,0.562500,0.500000,0.770833,0.000000,0.076190,0.000000,0.225000,0.000000,0.062500,0.000000,0.104167


In [10]:
### MLP
#[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, 
# (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-ARTICULO/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, accuracy, roc_auc,
 (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-PRUEBA/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.37.pickle')

#p_micro
#r_micro
#f1_micro
#p_macro
#r_macro
#f1_macro
accuracy
#roc_auc

[0.5, 0.6666666666666666]

# Probas varias

In [ ]:
# proba MLP
#z=FileUtility.load_obj('../../crohns_disease/results/DNNclassifier/weights_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')
#t=FileUtility.load_sparse_csr('../../crohns_disease/results/DNNclassifier/activationlayer_layers_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

#print(z)

###########################################################################################

#[label_set, conf, best_score_, best_estimator_, cv_results_, best_params_, predictions]=FileUtility.load_obj('../../crohns_disease/results-PRUEBA/RFclassifier/all_results.pickle')

#results=pd.read_csv("../../crohns_disease/results-PRUEBA/RFclassifier/cv_results_.csv")

#results.columns

#results[['mean_test_f1_macro', 'mean_test_f1_micro', 'mean_test_precision_macro', 
#       'mean_test_precision_micro', 'mean_test_recall_macro', 'mean_test_recall_micro',
#       'std_test_f1_macro', 'std_test_f1_micro', 'std_test_precision_macro',
#       'std_test_precision_micro', 'std_test_recall_macro', 'std_test_recall_micro']]


#results.drop(['mean_fit_time', 'mean_score_time', 'std_fit_time', 'std_score_time', 
#              'param_max_features', 'param_min_samples_leaf', 'param_min_samples_split',
#              'param_criterion', 'param_n_estimators', 'rank_test_f1_macro', 'rank_test_f1_micro',
#              'rank_test_precision_macro', 'rank_test_precision_micro', 'rank_test_recall_macro',
#              'rank_test_recall_micro'], axis=1)

###########################################################################################

#[latex_line, p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, 
# (loss_values, val_loss_values, epochs)]=FileUtility.load_obj('../../crohns_disease/results-ARTICULO/DNNclassifier/results_mlp_1024-0.2-512-0.2-256-0.1-128-8_0.65.pickle')

###########################################################################################3

#print(sys.version)
#import pkg_resources
#installed_packages = pkg_resources.working_set
#installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
#for i in installed_packages])
#print(installed_packages_list)